# Fake data creator for the DB Project
This was copied from something I had done for an earlier assignment.

In [1]:
!pip install Faker

In [4]:
"""
Created on: February 14, 2024

Author: Tuck Williamson

Assignment: 601 Misc\Library
Description: Library Faker

Library Faker
=============
This module creates fake data for testing purposes.
"""
from faker import Faker
import typing
from IPython.display import Markdown

Faker.seed(601)
fake = Faker('en_US')

sqte = "'"
sqteJoin = ", "


def create_sql_insert(elements: typing.Iterable[typing.Iterable[str]], table: str,
                      fields: typing.Iterable[str] = None) -> str:
    """
    This creates a **SQL INSERT** statement given the elements and table name.
    Fields can also be specified as an array of strings in the order they
    were placed in the tuples in the elements list.

    :param elements: The elements to inject as values into the table. Any elements that
    :param table: The name of the table to insert into.
    :param fields: An optional list of fields names in the order that they were placed in the elements list items.
    :return: A string containing the sql for the insert statement.

    """
    return f'```SQL\n\nINSERT INTO {table}{"" if fields is None else " (" + ", ".join(fields) + ")"}\n\tVALUES' + ', \n'.join(
        [f"({sqteJoin.join(ele)})" for ele in
         [[sqte + ele + sqte if not ele.startswith('raw: ') else ele[5:] for ele in etuple] for etuple in
          elements]]) + ";\n\n```"


# This will hold our final SQL string.
# sqlOut: str = ""

pubNum = 3

pubs = [(fake.company(), fake.address(), fake.numerify('###-###-####')) for i in range(pubNum)]
display(Markdown(create_sql_insert(pubs, "LIBRARY.PUBLISHER")))

# Pull the key -> name out to randomize.
pubKeys = [pub[0] for pub in pubs]

bookNum = 3
books = [(fake.catch_phrase(), fake.random_element(elements=pubKeys)) for book in range(bookNum)]
display(Markdown(create_sql_insert(books, "LIBRARY.BOOK", fields=["title", "publisher_name"])))


def make_select_for_key(key: str, table: str, column: str = None, value: str = None, where_clause: str = None) -> str:
    """
    This creates a **SQL SELECT** statement from the given parameters.
    These are generally used inside the SQL INSERT statements created from create_sql_insert.
    :param key: The name of the column to select, or a **string** with the name of multiple columns to
        select separated by commas.
    :param table: The name of the table to select from.
    :param column: The name of the column to check against value.
    :param value: The value to check against. See _whereClause_.
    :param where_clause: If whereClause is specified and value and column are None then the whereClause will
        be inserted into the statement as is. This is useful when you need to check multiple values.
    :return: A string with the select statement.
    """
    if value is None and where_clause is None:
        raise Exception("Cannot have value and whereClause both None. Either specify value or whereClause.")
    if column is None and value is not None:
        column = key

    if column is None and value is None:
        return f"raw: (SELECT {key} FROM {table} WHERE {where_clause})"
    else:
        return f"raw: (SELECT {key} FROM {table} WHERE {column} = '{value}')"


bookWhere = [book[0] for book in books]
authors = [(fake.name(), make_select_for_key('book_id', 'library.book', column='title', value=book)) for book in
           bookWhere]

display(Markdown(create_sql_insert(authors, "LIBRARY.BOOK_AUTHORS", fields=["author_name", "book_id"])))

branchNum = 3
branches = [(fake.name() + " Branch", fake.address()) for branch in range(3)]
branchWhere = [branch[0] for branch in branches]

display(Markdown(create_sql_insert(branches, "LIBRARY.LIBRARY_BRANCH", fields=["branch_name", "address"])))

# Now create some books to check out.
# Will start with some popular titles (every branch has some).
popularBooks = bookWhere[:(bookNum // 3)]
# Less popular (1-2) have copies.

lessPopularBooks = bookWhere[(bookNum // 3):bookNum - (bookNum // 3)]
# And finally 1 random branch has a copy
unPopularBooks = bookWhere[bookNum - (bookNum // 3):]
assert bookNum == len(popularBooks) + len(lessPopularBooks) + len(unPopularBooks)

copies = []
library_branch = 'library.library_branch'
library_book = 'library.book'
[
    copies.extend(
        [
            (str(fake.random_int(min=1, max=5)),
             make_select_for_key('book_id', library_book, column='title', value=book),
             make_select_for_key('branch_id', library_branch, column='branch_name', value=branch)) for branch
            in branchWhere
        ]) for book in popularBooks
]
# This one has a random number of branches and fewer copies.
[
    copies.extend(
        [
            (str(fake.random_int(min=1, max=3)),
             make_select_for_key('book_id', library_book, column='title', value=book),
             make_select_for_key('branch_id', library_branch, column='branch_name', value=branch)) for branch
            in fake.random_elements(branchWhere, unique=True)
        ]) for book in lessPopularBooks
]
# This one has a branch each and fewer copies.
[
    copies.append(
        (str(fake.random_int(min=1, max=2)),
         make_select_for_key('book_id', library_book, column='title', value=book),
         make_select_for_key('branch_id', library_branch, column='branch_name', value=fake.random_element(branchWhere)))
    ) for book in unPopularBooks
]
display(Markdown(create_sql_insert(copies, "LIBRARY.BOOK_COPIES", fields=["no_of_copies", "book_id", "branch_id"])))

borrowNumbers = 3
borrowers = [(fake.name(), fake.address(), fake.numerify('###-###-####')) for borrower in range(borrowNumbers)]

display(Markdown(create_sql_insert(borrowers, "LIBRARY.BORROWER", fields=["name", "address", "phone"])))


def get_rnd_date(future_date: bool = False) -> str:
    """
    This generates a random date offset from today. If future_date is true, it will be a date in the future.

    :param future_date: If true, it will be a date in the future.
    :return: String with random date for a SQL insertion.
    """
    return "raw: now() {} '{} {}'::interval".format('+' if future_date else '-', fake.random_element(
        [1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 4, 4, 3]),
                                                    fake.random_element(['day', 'month', 'week']))


bookLoans = [(book_copies[1],
              book_copies[2],
              make_select_for_key('card_no', 'library.borrower',
                                  where_clause="name = '{0}' and address = '{1}'".format(
                                      *fake.random_element(borrowers))), get_rnd_date(), get_rnd_date(True)) for
             book_copies in
             copies]
# I could randomly switch that future boolean on the second get_rnd_date to have some books overdue.

display(Markdown(create_sql_insert(bookLoans, "LIBRARY.book_loans",
                            fields=["book_id", "branch_id", "card_no", 'Date_out', 'Due_date'])))

# sqlOut = sqlOut.replace("'(", "(").replace(")'", ")")

```SQL

INSERT INTO LIBRARY.PUBLISHER
	VALUES('Jennings PLC', '6700 Stevens Shoals
Elizabethborough, IA 62697', '584-633-7063'), 
('Smith, Nunez and Myers', '478 Christopher Gateway
West Debramouth, NV 28645', '900-535-0619'), 
('Peterson Ltd', '54984 Ferguson Dam
Denisefort, NV 85984', '770-293-3838');

```

```SQL

INSERT INTO LIBRARY.BOOK (title, publisher_name)
	VALUES('Face-to-face mobile archive', 'Peterson Ltd'), 
('Team-oriented 5thgeneration paradigm', 'Jennings PLC'), 
('Innovative zero administration support', 'Jennings PLC');

```

```SQL

INSERT INTO LIBRARY.BOOK_AUTHORS (author_name, book_id)
	VALUES('Anita Jones', (SELECT book_id FROM library.book WHERE title = 'Face-to-face mobile archive')), 
('Timothy Landry', (SELECT book_id FROM library.book WHERE title = 'Team-oriented 5thgeneration paradigm')), 
('Richard Hall', (SELECT book_id FROM library.book WHERE title = 'Innovative zero administration support'));

```

```SQL

INSERT INTO LIBRARY.LIBRARY_BRANCH (branch_name, address)
	VALUES('Carrie Hammond Branch', '2370 Megan Lakes Apt. 526
Lake Karenborough, VI 86533'), 
('Melissa Palmer Branch', '7032 Hannah Villages Suite 972
Nicoleborough, IL 07129'), 
('Michael Barnes Branch', '6431 Scott Tunnel Suite 332
Alexishaven, KS 97971');

```

```SQL

INSERT INTO LIBRARY.BOOK_COPIES (no_of_copies, book_id, branch_id)
	VALUES('2', (SELECT book_id FROM library.book WHERE title = 'Face-to-face mobile archive'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Carrie Hammond Branch')), 
('4', (SELECT book_id FROM library.book WHERE title = 'Face-to-face mobile archive'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Melissa Palmer Branch')), 
('4', (SELECT book_id FROM library.book WHERE title = 'Face-to-face mobile archive'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Michael Barnes Branch')), 
('3', (SELECT book_id FROM library.book WHERE title = 'Team-oriented 5thgeneration paradigm'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Carrie Hammond Branch')), 
('1', (SELECT book_id FROM library.book WHERE title = 'Innovative zero administration support'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Melissa Palmer Branch'));

```

```SQL

INSERT INTO LIBRARY.BORROWER (name, address, phone)
	VALUES('Steven Jenkins', '381 Williams Views Suite 279
Garciahaven, TN 73228', '176-010-6225'), 
('Lindsey Hanson', '10251 Kristen Radial Apt. 698
Hickmanberg, NV 80983', '928-525-7549'), 
('Michael Johnson', '27966 Anderson Village Apt. 382
Lake John, AL 64436', '522-667-9368');

```

```SQL

INSERT INTO LIBRARY.book_loans (book_id, branch_id, card_no, Date_out, Due_date)
	VALUES((SELECT book_id FROM library.book WHERE title = 'Face-to-face mobile archive'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Carrie Hammond Branch'), (SELECT card_no FROM library.borrower WHERE name = 'Lindsey Hanson' and address = '10251 Kristen Radial Apt. 698
Hickmanberg, NV 80983'), now() - '3 day'::interval, now() + '1 month'::interval), 
((SELECT book_id FROM library.book WHERE title = 'Face-to-face mobile archive'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Melissa Palmer Branch'), (SELECT card_no FROM library.borrower WHERE name = 'Lindsey Hanson' and address = '10251 Kristen Radial Apt. 698
Hickmanberg, NV 80983'), now() - '1 month'::interval, now() + '1 month'::interval), 
((SELECT book_id FROM library.book WHERE title = 'Face-to-face mobile archive'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Michael Barnes Branch'), (SELECT card_no FROM library.borrower WHERE name = 'Michael Johnson' and address = '27966 Anderson Village Apt. 382
Lake John, AL 64436'), now() - '3 month'::interval, now() + '1 month'::interval), 
((SELECT book_id FROM library.book WHERE title = 'Team-oriented 5thgeneration paradigm'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Carrie Hammond Branch'), (SELECT card_no FROM library.borrower WHERE name = 'Steven Jenkins' and address = '381 Williams Views Suite 279
Garciahaven, TN 73228'), now() - '4 month'::interval, now() + '3 month'::interval), 
((SELECT book_id FROM library.book WHERE title = 'Innovative zero administration support'), (SELECT branch_id FROM library.library_branch WHERE branch_name = 'Melissa Palmer Branch'), (SELECT card_no FROM library.borrower WHERE name = 'Steven Jenkins' and address = '381 Williams Views Suite 279
Garciahaven, TN 73228'), now() - '1 day'::interval, now() + '3 month'::interval);

```